In [1]:
from snowflake_utils import (
    compare_bodo_and_snowflake_outputs,
    execute_query_and_save_to_snowflake_table,
    get_snowflake_connection,
)

In [2]:
# Create connection object.
con = get_snowflake_connection(
    "creds.json", database="SNOWFLAKE_SAMPLE_DATA", warehouse="DEMO_WH"
)

/opt/anaconda3/envs/bodo_dev_310/lib/python3.10/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [3]:
# Running arbitrary query, e.g. to check how much data would we end up with after applying a filter
res = con.cursor().execute(
    "select count(*) from SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER where C_MKTSEGMENT = 'HOUSEHOLD'"
)
res.fetchall()

[(30189,)]

In [4]:
# Read the user query from the file:
with open("tpch_q1.sql", "r") as f:
    query = f.read()

In [5]:
# Generate the output of the working query in Snowflake and write it to a table
execute_query_and_save_to_snowflake_table(
    con, query, "TEST_DB.SNOWFLAKE_WRITE_TEST.TPCH_Q1_SF_OUT"
)

[('Table TPCH_Q1_SF_OUT successfully created.',)]

In [6]:
# Run the query in Bodo using the BodoSQLWrapper and write the output to a Snowflake table
# e.g. On Platform, run this command the terminal:
# px python -u BodoSQLWrapper.py -c creds.json -f tpch_q1.sql -w DEMO_WH -d SNOWFLAKE_SAMPLE_DATA --sf_out_table_loc TEST_DB.SNOWFLAKE_WRITE_TEST.TPCH_Q1_BODO_OUT

In [7]:
# Compare Snowflake output with Bodo output (no output means it matched)
# In this case, there are minor differences due to the float columns.
compare_bodo_and_snowflake_outputs(
    con,
    "TEST_DB.SNOWFLAKE_WRITE_TEST.TPCH_Q1_BODO_OUT",
    "TEST_DB.SNOWFLAKE_WRITE_TEST.TPCH_Q1_SF_OUT",
)

/opt/anaconda3/envs/bodo_dev_310/lib/python3.10/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,N,F,991417.0,1.487505e+09,1.413082e+09,1.469649e+09,25.516472,38284.467761,0.050093,38854
1,N,O,76633518.0,1.149352e+11,1.091896e+11,1.135610e+11,25.502020,38248.015609,0.050000,3004998
2,R,F,37719753.0,5.656804e+10,5.374129e+10,5.588962e+10,25.505794,38250.854626,0.050009,1478870
3,A,F,37734107.0,5.658655e+10,5.375826e+10,5.590907e+10,25.522006,38273.129735,0.049985,1478493
